In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid , save_image
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier

In [ ]:
def get_data():
    used_data = pd.read_csv('../data/data.csv', index_col=0, parse_dates=True)
    return used_data

def get_all_tickers():
    used_data = get_data()
    return used_data.columns

def get_returns():
    used_data = get_data()
    ret_day = ret_day = used_data.fillna(0)
    S = np.exp(used_data.fillna(0)).cumprod()
    ret_mon = S.resample('M').ffill().pct_change().dropna() # start 2000-02-29
    return ret_day, ret_mon
    
def form_labels(tic):
    ret_day, ret_mon = get_returns()
    median_ret = ret_mon.median(1)
    labels = ret_mon.shift(-1).gt(median_ret.shift(-1), axis=0).astype(int)[tic]
    labels.name = 'label'
    return labels

In [ ]:
def form_features(tic):
    ret_day, ret_mon = get_returns()
    
    feature_data = []
        
    daily = ret_day[tic]
    dfs = []
    idx = range(13, len(ret_mon) - 1)
    for t in idx:
        current_y = ret_mon.index[t].year
        current_mon = ret_mon.index[t].month
        next_mon_isjan = (ret_mon.index[t+1].month == 1)

        start = t - 13
        end = t - 2

        ret_mon_slice = ret_mon[tic].iloc[start:end+1]

        last_date = (daily[(daily.index.year == current_y) 
                           & (daily.index.month == current_mon)].last_valid_index())

        last_date_loc = daily.index.get_loc(last_date)
        start_date_loc = last_date_loc - 19

        daily_rets_slice = daily.iloc[start_date_loc:last_date_loc + 1]

        daily_rets_slice.index = ['R_d_' + str(i) for i in range(1, 21)]
        ret_mon_slice.index = ['R_m_' + str(i) for i in range(1, 13)]

        conc = pd.concat([daily_rets_slice, ret_mon_slice])
        conc['JAN'] = next_mon_isjan

        conc = conc.to_frame()
        conc.columns = [t]
        dfs.append(conc)
    feats_t = pd.concat(dfs, axis=1).T
    feature_data.append(feats_t)
    
    date_idx = ret_mon.index[idx]
    feature_data_df = pd.concat(feature_data, axis=0).set_index(date_idx)
    return feature_data_df

In [ ]:
def form_feats_labels(tickers):
    full_data_by_tic = []
    for tic in tickers:
        feats = form_features(tic)
        labels = form_labels(tic)
        conc = pd.concat([feats, labels], axis=1, join='inner')
        full_data_by_tic.append(conc)
    return pd.concat(full_data_by_tic, axis=0) # datetime index not unique

In [ ]:
train_size = 0.7
def split_test_train(train_set_size=train_size):
    tickers = get_all_tickers()
    Xtrains = []
    ytrains = []
    Xtests = []
    ytests = []
    for tic in tickers:
        data = form_feats_labels([tic])
        X = data[data.columns[:-1]].as_matrix()
        y = data.label.as_matrix()
        N = len(y)
        train_set_stop = int(N * train_set_size)
        
        Xtrain = X[0:train_set_stop]
        Xtrains.append(Xtrain)
        ytrain = y[0: train_set_stop]
        ytrains.append(ytrain)
        
        Xtest = X[train_set_stop:]
        Xtests.append(Xtest)
        ytest = y[train_set_stop:]
        ytests.append(ytest)
    
    Xtrains_c = np.concatenate(Xtrains)
    ytrains_c = np.concatenate(ytrains)
    Xtests_c = np.concatenate(Xtests)
    ytests_c = np.concatenate(ytests)
    return Xtrains_c, ytrains_c, Xtests_c, ytests_c      

In [ ]:
class FutureDataset(torch.utils.data.Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(33, 50)
        self.fc21 = nn.Linear(50, 4)
        self.fc22 = nn.Linear(50, 4)
        self.fc3 = nn.Linear(4, 50)
        self.fc4 = nn.Linear(50, 33)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5*logvar)
            eps = Variable(torch.rand(std.size()), requires_grad=False)
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return F.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 33))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
        
    def loss_function(self, recon_x, x, mu, logvar):
        MSE = F.mse_loss(recon_x, x.view(-1, 33), size_average=False)

        # see Appendix B from VAE paper:
        # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
        # https://arxiv.org/abs/1312.6114
        # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
        KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        return MSE + KLD

In [ ]:
# Train and test data
Xtrain, ytrain, Xtest, ytest = split_test_train()

batch_size = 64
train_dataset = FutureDataset(Xtrain, ytrain)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size)

test_dataset = FutureDataset(Xtest, ytest)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

In [ ]:
log_interval = 100
number_of_epochs = 500

vae = VAE()
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

In [ ]:
vae.train()
for epoch in range(number_of_epochs):
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = Variable(data.float())
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(data)
        loss = vae.loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.data[0]
        optimizer.step()
    
    if (epoch+1) % log_interval == 0:
        print('====> Epoch: {} Average loss: {:.4f}'.format(
              epoch, train_loss / len(train_loader.dataset)))

In [ ]:
vae.eval()
test_loss = 0
for i, (data, _) in enumerate(test_loader):
    data = Variable(data.float(), requires_grad=False)
    recon_batch, mu, logvar = vae(data)
    test_loss += vae.loss_function(recon_batch, data, mu, logvar).data[0]

test_loss /= len(test_loader.dataset)
print('====> Test set loss: {:.4f}'.format(test_loss))

In [ ]:
# Build encoded dataset
Xenc = np.array([]).reshape(0, 4)
yenc = np.array([])
for _, (data,target) in enumerate(train_loader):
    data = Variable(data.view(-1,33)).float()
    labels = Variable(target).type(torch.LongTensor)
    
    _, data, _ = vae(data)
    
    batch = data.cpu().data.numpy()
    Xenc = np.vstack((Xenc, batch))
    yenc = np.append(yenc, target.numpy())

In [ ]:
# RANDOM FOREST CLASSIFIER
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(Xenc, yenc)
score = cross_val_score(clf, Xenc, yenc)
print(score.mean())

print("What features are prefered", clf.feature_importances_)

In [ ]:
# NAIVE BAYESIAN CLASSIFIER
gnb = GaussianNB()
gnb.fit(Xenc, yenc)
score = cross_val_score(gnb, Xenc, yenc)
print(score.mean())

In [ ]:
# ORGINAL DATA
tickers = get_all_tickers()
X_and_y = form_feats_labels(tickers)
Xorg = X_and_y[X_and_y.columns[:-1]].as_matrix()
yorg = X_and_y.label.as_matrix()

In [ ]:
# MLP ONLY
mlp = MLPClassifier(activation='logistic',
                    batch_size=32,
                    hidden_layer_sizes=[5, 5],
                    alpha=0.1)
mlp.fit(Xtrain, ytrain)
score = cross_val_score(clf, Xtest, ytest)
print(score.mean())

In [ ]:
# RANDOM FOREST CLASSIFIER ONLY
clf = RandomForestClassifier(max_depth=20, random_state=0)
clf.fit(Xtrain, ytrain)
score = cross_val_score(clf, Xtest, ytest)
print(score.mean())

print("What features are prefered: \n", clf.feature_importances_)

In [ ]:
# ADABOOST ONLY
ada = AdaBoostClassifier(n_estimators=10)
ada.fit(Xorg, yorg)  
score = cross_val_score(ada, Xorg, yorg)
print(score.mean())

In [ ]:
# SVC ONLY
svc = SVC(gamma=2, C=1)
svc.fit(Xorg, yorg)
score = cross_val_score(svc, Xorg, yorg)
print(score.mean())

In [ ]:
# BROKEN, PREDICTION RESULTS FROM ENCODER PIPELINE
prediction_outputs = []
returns = []

classifier = clf

for idx, observation in enumerate(Xorg):
    data = torch.from_numpy(observation)

    data = Variable(data).float()
    _, data, _ = vae(data)
    output = classifier.predict_proba(data.cpu().data.numpy())
    prediction_outputs.append(output[0])

len(prediction_outputs)

In [ ]:
def get_positions(prediction_outputs):
    result_df = pd.DataFrame(prediction_outputs, columns=["loser_p", "winner_p"])
    label = (result_df["winner_p"] > result_df["loser_p"]).astype(int) # <
    result_df["longshort"] = label.map({1: 1, 0: -1})

    print("Predicted long count:", sum(label))

    return result_df

In [ ]:
result_dfs = {}

for tic in ["ES", "FESX", "GC", "NK", "TU", "ED"]:
    test_start = int(len(X) * train_size) + 1
    X = form_features(tic).iloc[test_start:]
    prediction_outputs = clf.predict_proba(X)
    df = get_positions(prediction_outputs).set_index(X.index)
    result_dfs[tic] = df[['longshort']]

In [ ]:
ret_day, ret_mon = get_returns()
ret_mon_slice = ret_mon.iloc[13:]
for key in result_dfs.keys():
    newcol = str(key) + '_pos'
    ret_mon_slice[newcol] = result_dfs[key]
ret_mon_slice = ret_mon_slice.resample('M').ffill()

In [ ]:
leverage = 0.05/(ret_day.ewm(60).std()*np.sqrt(250))
leverage_mon = leverage.resample('M').ffill()

positions = ret_mon_slice[ret_mon_slice.columns[-6:]]
returns = ret_mon_slice[ret_mon_slice.columns[:6]]
positions.columns = ['ES', 'FESX', 'GC', 'NK', 'TU', 'ED']

position_returns = returns * positions.shift(1)
vol_adj_rets = (position_returns * leverage_mon.shift(1)).dropna()
total_rets = vol_adj_rets.mean(1)
(total_rets + 1).cumprod().plot(title='vola-adjusteering')
m = total_rets.mean() * 12
s = total_rets.std() * np.sqrt(12)
print ('Sharpe', m/s)
plt.show()